In [1]:
import os
import sys
import netCDF4 as nc
import numpy as np
import pickle as pk
from scipy.spatial import distance
import random

Download hail size data from 2021-06-20 event\
https://zenodo.org/records/10609730/files/Hail_size_data.zip?download=1

In [2]:
fn = 'Hail_size_data/Flight_01_FA_20210620.nc'
ds = nc.Dataset(fn)

In [3]:
major_ax = ds['hail_major_axis'][:].data
minor_ax =  ds['hail_minor_axis'][:].data
hail_co_x = ds['hail_center_x'][:].data
hail_co_y = ds['hail_center_y'][:].data

In [54]:
#Ground samppling distance [mm]
gsd = ds['gsd'][:].data

#Coordinates (in pixels) to place the virtual sensors within the orthophoto (600 m2)
xmin = 2400
xmax = 19393
ymin = 3700
ymax = 19393

#Number of virtual sensors
nsens = 100
sensor_area = 0.2 #m2

vsensor_dict = {}
vsens_co = []

vsensor_dict['dist'] = []
vsensor_dict['major'] = []
vsensor_dict['minor'] = []
vsensor_dict['hail_x'] = []
vsensor_dict['hail_y'] = []
    
for n in range(0,nsens):
    #Randomly place hailsensor (center coordinates) on field and save hits
    xrand = random.randrange(xmin,xmax)
    yrand = random.randrange(ymin,ymax)
    sens_co = (xrand, yrand)
    vsens_co.append(sens_co)
    
    dist_list = []
    major_list = []
    minor_list = []
    hail_x = []
    hail_y = []

    for i,x in enumerate(hail_co_x):
        y = hail_co_y[i]
        hail_co = (x,y)

        #Calculate distance from center of virtual sensor pixel to hail center pixels
        dist = distance.euclidean(sens_co, hail_co) * gsd
       
        #If distance is within the virtual sensor area count it as a hit
        if dist <= np.sqrt(sensor_area/np.pi)*1000:
            dist_list.append(dist)
            major_list.append(major_ax[i])
            minor_list.append(minor_ax[i])
            hail_x.append(x)
            hail_y.append(y)

    vsensor_dict['dist'].append(dist_list)
    vsensor_dict['major'].append(major_list)
    vsensor_dict['minor'].append(minor_list)
    vsensor_dict['hail_x'].append(hail_x)
    vsensor_dict['hail_y'].append(hail_y)

vsensor_dict['vsens_co'] = vsens_co

#Save dictionary:
with open('VSENSOR_N100_600m2.pkl','wb') as f:
    pk.dump(vsensor_dict, f)